In [2]:
import numpy as np
import networkx as nx
import json
from docplex.mp.model import Model

In [ ]:
def MIS_QUBO(G, penalty=2):
    # MIS model as a QUBO problem
    mdl = Model('MIS')
    num_vertices = G.number_of_nodes()

    x = {i: mdl.binary_var(name=f"x_{i}") for i in range(num_vertices)}
    mdl.minimize(-mdl.sum(x) + penalty * mdl.sum(
        x[i] * x[j] for (i, j) in G.edges
    ))
    return mdl

In [42]:
def kings_graph(numx, numy, filling=0.7, seed=None):
    '''
    Generate a next nearest neighbor graph with a lattice constant 1, with some number of nodes removed
    numx    - number of grid points in the X direction
    numy    - number of grid points in the Y direction
    filling - Fraction of vertices to be kept. Total number of vertices is int(numx*numy*filling)

    Returns
    pos     - [N x 2] array of points on a square grid
    graph   - networkx connectivity graph
    '''
    xx, yy = np.meshgrid(range(numx), range(numy))
    num_points = int(numx * numy * filling)
    rand = np.random.default_rng(seed=seed)
    # Generate points
    points = np.array([xx.flatten(), yy.flatten()]).T
    points = points[rand.permutation(numx * numy)[0:num_points], :]
    # Generate a unit disk graph by thresholding distances between points.
    distances = np.sqrt((points[:, 0] - points[:, 0, None]) ** 2 + (points[:, 1] - points[:, 1, None]) ** 2)
    graph = nx.Graph(distances < np.sqrt(2) + 1E-10)

    graph.remove_edges_from(nx.selfloop_edges(graph))
    return points, graph, distances

In [48]:
a = 6.78806e-6

lattice = 'graph'
seed = 123
grid_side_list = range(4,40)
filling_list = [0.7,0.85]

for grid_side in grid_side_list:
    for filling in filling_list:
        numx,numy = [grid_side,grid_side]
        pos, G, distances = kings_graph(numx, numy, filling, seed)
        nq = G.number_of_nodes()
        if nq < 1000:
            print(f"Number of qubits: {nq}")
            mdl = MIS_QUBO(G)
            mdl.solve()
            sol = "".join(str(round(mdl.solution.get_value(var))) for var in mdl.iter_binary_vars())

            problem = {}
            problem["pos"] = list([list([round(j) for j in i]) for i in pos])
            problem["edges"] = list(G.edges())
            problem["distances"] = list([list([float(j) for j in i]) for i in distances])
            problem["sol"] = sol
            problem["grid_side"] = grid_side
            with open(f"./Data/problems/{nq}.json", "w") as file:
                json.dump(problem, file)

Number of qubits: 11
Number of qubits: 13
Number of qubits: 17
Number of qubits: 21
Number of qubits: 25
Number of qubits: 30
Number of qubits: 34
Number of qubits: 41
Number of qubits: 44
Number of qubits: 54
Number of qubits: 56
Number of qubits: 68
Number of qubits: 70
Number of qubits: 85
Number of qubits: 84
Number of qubits: 102
Number of qubits: 100
Number of qubits: 122
Number of qubits: 118
Number of qubits: 143
Number of qubits: 137
Number of qubits: 166
Number of qubits: 157
Number of qubits: 191
Number of qubits: 179
Number of qubits: 217
Number of qubits: 202
Number of qubits: 245
Number of qubits: 226
Number of qubits: 275
Number of qubits: 252
Number of qubits: 306
Number of qubits: 280
Number of qubits: 340
Number of qubits: 308
Number of qubits: 374
Number of qubits: 338
Number of qubits: 411
Number of qubits: 370
Number of qubits: 449
Number of qubits: 403
Number of qubits: 489
Number of qubits: 437
Number of qubits: 531
Number of qubits: 473
Number of qubits: 574
Num

## QAOA schedule used for the parameter transfer

In [51]:
omega_opt_list = np.array([0.0013533659033773317,
 4.045866754699931,
 2.9797945552295904,
 4.031646387060219,
 6.13680201056113,
 5.042842461864641,
 2.9761309901447177,
 1.8335073028710545,
 2.8312315456027655,
 0.005323555828519899])*1e6 //400 * 400

delta_opt_list = np.array([-2.2705852238763873,
 -10.267156003603997,
 -1.37474817026258,
 0.0644988201504712,
 2.869285157654999,
 6.309152585006643,
 0.7500851533315123,
 15.27993355344018,
 -6.074855874393538,
 -21.428780809189433])*1e6 //400 * 400

with open("./Data/opt_schedule.json", "w") as file:
    json.dump({"omega_list":[float(i) for i in omega_opt_list],
               "delta_list":[float(i) for i in delta_opt_list]}, file)